In [ ]:
# runs in jupyter container
import os
import torch
import pandas as pd
from torch.utils.data import DataLoader, Dataset

DATA_DIR = os.getenv("MOVIELENS_DATA_DIR", "/mnt/Project-37")
print("Data dir:", DATA_DIR)

In [ ]:
# runs in jupyter container
eval_path = os.path.join(DATA_DIR, "evaluation", "movielens_192m_eval.txt")
print("Evaluation file path:", eval_path)

eval_df = pd.read_csv(eval_path, sep="\t", names=["userId", "movieId"], nrows=10_000)
print(eval_df.head())

In [ ]:
# runs in jupyter container
class EvalDataset(Dataset):
    def __init__(self, dataframe):
        self.users = torch.tensor(dataframe['userId'].values, dtype=torch.long)
        self.items = torch.tensor(dataframe['movieId'].values, dtype=torch.long)

    def __len__(self):
        return len(self.users)

    def __getitem__(self, idx):
        return self.users[idx], self.items[idx]

eval_dataset = EvalDataset(eval_df)
eval_loader = DataLoader(eval_dataset, batch_size=128, shuffle=False)


In [ ]:
# runs in jupyter container
model_path = "models/SSE_PT.pth"
model = torch.load(model_path, map_location="cpu")
model.eval()
print("Model loaded.")

In [ ]:
# Run a simple forward pass
results = []

with torch.no_grad():
    for i, (users, items) in enumerate(eval_loader):
        outputs = model(users, items)
        results.extend(outputs.cpu().numpy())
        if i % 10 == 0:
            print(f"Processed batch {i}, output shape: {outputs.shape}")